In [1]:
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras.callbacks import *
from tensorflow.keras import backend as K
from __TargetEmbedding__ import auto_delete_hdf5_save_history, Encoder, Decoder
from __init__ import glob_all
from RandomTesting.__RandomTesting__ import RandomArea_Testing, calculate_extract_loss
from tqdm import tqdm

import tensorflow.compat.v1 as tf
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
import keras
import os


warnings.filterwarnings('ignore')
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
tf.config.optimizer.set_jit(True)

Using TensorFlow backend.


In [2]:
channel_number = 6; train_bs = 12; total_training_time = 30

EPA73_test_pm25 = np.load('EPA73_Testing_348X204_PM25.npy', mmap_mode='r')
station_coordinate = pd.read_csv('RandomTesting/station_coordinate')
exArea_KNN = np.load('RandomTesting/EPA73_testing_KNN3fill_area.npy', mmap_mode='r')

AIR73_train = np.load('AIR_DomainA_Training.npy', mmap_mode='r')
AIR_train = np.load('AIR_DomainB_Training.npy', mmap_mode='r')

AIR73_test = np.load('AIR_DomainA_Testing.npy', mmap_mode='r')
AIR_test = np.load('AIR_DomainB_Testing.npy', mmap_mode='r')

SAT73_train = np.load('SAT_DomainA_Training.npy', mmap_mode='r')
SAT_train = np.load('SAT_DomainB_Training.npy', mmap_mode='r')

SAT73_test = np.load('SAT_DomainA_Testing.npy', mmap_mode='r')
SAT_test = np.load('SAT_DomainB_Testing.npy', mmap_mode='r')


train73 = np.concatenate([AIR73_train,SAT73_train])
test73 = np.concatenate([AIR73_test,SAT73_test])
train = np.concatenate([AIR_train,SAT_train])
test = np.concatenate([AIR_test,SAT_test])


ALL_Encoder_model = Encoder(channel_number)
ALL_Decoder_model = Decoder(channel_number)
x = Input(shape=(348, 204, 1))
ALL_Autoencoder = Model(x,ALL_Decoder_model(ALL_Encoder_model(x)))
Unpair_weight = glob_all('Weight/AIRSAT_channel'+str(channel_number))[0]
ALL_Autoencoder.load_weights(Unpair_weight)
for layer in ALL_Autoencoder.layers:layer.trainable = False
for layer in ALL_Autoencoder.layers:layer.trainable = False
for layer in ALL_Autoencoder.layers:layer.trainable = False

In [ ]:
area_list = ['Taipei_area', 'Zhumiao_area', 'Taichung_area', 'YunChiNan_area', 'KaoPing_area']
for training_time in range(total_training_time):
    K.clear_session()
           
    path ='Unpair_Target_Embedding_AIRSAT_channel'+str(channel_number)+'_K3Area/'+str(training_time)+'/'
    if not os.path.exists(path):os.makedirs(path)    
    
    Unpair73_Encoder_model = Encoder(channel_number)
    Unpair73_Autoencoder = Model(x,ALL_Decoder_model(Unpair73_Encoder_model(x)))
    Unpair73_Autoencoder.compile(loss='mse',optimizer='adam')    

    filepath=path+'{epoch:04d}-train_{loss:f}-test_{val_loss:f}.h5'
    earlystopper = EarlyStopping(monitor='val_loss', patience=120, min_delta=0.01, verbose=0)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor = 0.2, patience=30, min_lr = 2e-7, verbose = 0)   

    checkpoint = ModelCheckpoint(filepath, monitor='val_loss', save_best_only=True, mode='auto')    
    callbacks_list = [earlystopper,reduce_lr,checkpoint]   

    history = Unpair73_Autoencoder.fit(train73,train, validation_data=(test73, test),
                                    batch_size = train_bs, epochs=2000,
                                    callbacks = callbacks_list, verbose=2)      
    
    auto_delete_hdf5_save_history(history,path)
    Unpair73_Autoencoder.load_weights(glob_all(path)[0])
    
    total_mse, total_mae, total_mape = ([] for _ in range(3))

    for i, area in enumerate(area_list):
        a, lats_ex, lons_ex = RandomArea_Testing(area, station_coordinate, EPA73_test_pm25)
        predict_test = Unpair73_Autoencoder.predict(exArea_KNN[i])
        mse, mae, mape = calculate_extract_loss(predict_test, EPA73_test_pm25, lats_ex, lons_ex)
        total_mse.append(mse); total_mae.append(mae); total_mape.append(mape)
    del predict_test; del a
    print('exarea-- mean mse:%.4f_mean mae:%.4f_mean mape:%.4f' %(np.mean(total_mse),np.mean(total_mae),np.mean(total_mape)))

    f1 = open(path+'Extractresult_Area.txt', 'a')
    print('----------'+'total_loss'+'----------',file=f1)
    print('mse:  '+str(np.round(np.array(total_mse),4)),file=f1)
    print('mae:  '+str(np.round(np.array(total_mae),4)),file=f1)
    print('mape: '+str(np.round(np.array(total_mape),4)),file=f1)
    print('---------------------------',file=f1)    
    
    print('----------'+'avg_mse_loss'+'----------',file=f1)
    print('max:  '+str(np.array(total_mse).max()),file=f1)
    print('min:  '+str(np.array(total_mse).min()),file=f1)
    print('mean: '+str(np.array(total_mse).mean()),file=f1)
    print('var:  '+str(np.array(total_mse).var()),file=f1)
    print('std:  '+str(np.array(total_mse).std()),file=f1)
    print('---------------------------',file=f1)

    print('----------'+'avg_mae_loss'+'----------',file=f1)
    print('max:  '+str(np.array(total_mae).max()),file=f1)
    print('min:  '+str(np.array(total_mae).min()),file=f1)
    print('mean: '+str(np.array(total_mae).mean()),file=f1)
    print('var:  '+str(np.array(total_mae).var()),file=f1)
    print('std:  '+str(np.array(total_mae).std()),file=f1)
    print('---------------------------',file=f1)

    print('----------'+'avg_mape_loss'+'----------',file=f1)
    print('max:  '+str(np.array(total_mape).max()),file=f1)
    print('min:  '+str(np.array(total_mape).min()),file=f1)
    print('mean: '+str(np.array(total_mape).mean()),file=f1)
    print('var:  '+str(np.array(total_mape).var()),file=f1)
    print('std:  '+str(np.array(total_mape).std()),file=f1)
    print('---------------------------',file=f1)         
    f1.close()

Epoch 1/2000
267/267 - 10s - loss: 163.8086 - val_loss: 40.0854 - lr: 0.0010
Epoch 2/2000
267/267 - 13s - loss: 122.1744 - val_loss: 47.8522 - lr: 0.0010
Epoch 3/2000
267/267 - 7s - loss: 120.9915 - val_loss: 43.0202 - lr: 0.0010
Epoch 4/2000
267/267 - 7s - loss: 114.0634 - val_loss: 44.0091 - lr: 0.0010
Epoch 5/2000
267/267 - 7s - loss: 112.4523 - val_loss: 43.9267 - lr: 0.0010
Epoch 6/2000
267/267 - 7s - loss: 109.4049 - val_loss: 39.4364 - lr: 0.0010
Epoch 7/2000
267/267 - 7s - loss: 107.0164 - val_loss: 41.9954 - lr: 0.0010
Epoch 8/2000
267/267 - 7s - loss: 103.4232 - val_loss: 37.3750 - lr: 0.0010
Epoch 9/2000
267/267 - 7s - loss: 102.7309 - val_loss: 37.4769 - lr: 0.0010
Epoch 10/2000
267/267 - 7s - loss: 100.6765 - val_loss: 43.8132 - lr: 0.0010
Epoch 11/2000
267/267 - 7s - loss: 99.5078 - val_loss: 39.0365 - lr: 0.0010
Epoch 12/2000
267/267 - 7s - loss: 97.0595 - val_loss: 39.3929 - lr: 0.0010
Epoch 13/2000
267/267 - 7s - loss: 96.5883 - val_loss: 37.8805 - lr: 0.0010
Epoch 14/